# Manual Implementation of Problem 2 (Regression)

## Importing Libraries

In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Setting up the data


### Loading and splitting dataset

In [12]:
df = pd.read_csv("California_Houses.csv")

df = df.sample(frac=1, random_state=42).reset_index(drop=True)

print(df.isna().sum())

train_data_size = int(0.7 * len(df))
validation_data_size = int(0.15 * len(df))
test_data_size = len(df) - train_data_size - validation_data_size

train_data = df[:train_data_size]
validation_data = df[train_data_size:train_data_size + validation_data_size]
test_data = df[train_data_size + validation_data_size:]

Median_House_Value          0
Median_Income               0
Median_Age                  0
Tot_Rooms                   0
Tot_Bedrooms                0
Population                  0
Households                  0
Latitude                    0
Longitude                   0
Distance_to_coast           0
Distance_to_LA              0
Distance_to_SanDiego        0
Distance_to_SanJose         0
Distance_to_SanFrancisco    0
dtype: int64


### Extracting features matrix & Target Vector

In [13]:
x = train_data.iloc[:, 1:13]
t = train_data.iloc[:, 0]

X_vectors = {}
for col in x.columns:
    X_vectors[col] = x[col].values.reshape(-1, 1)

X = np.hstack(list(X_vectors.values()))
T = t.values.flatten()


X_mean = np.mean(X, axis=0)
X_std = np.std(X, axis=0)
X_scaled = (X - X_mean) / X_std

n_samples = X.shape[0]
X_biased = np.hstack((np.ones((n_samples, 1)), X_scaled))

## Linear Regression Models

### Direct matrix Solution

$w^* = (X^{\top} X)^{-1} (X^{\top} T)$.

In [14]:
left_term = np.linalg.inv(X_biased.T @ X_biased)
right_term = X_biased.T @ T
W_star = left_term @ right_term
print("Learned Weights:", W_star)

Learned Weights: [207183.64947739  73339.23718052  11390.78283316  -9813.07726714
  39003.27192159 -50396.58892998  25030.66297081 -91070.56192613
 -58463.20851428 -12788.39669392 -34720.70712485  68437.02098175
   7981.2671976 ]


### Gradient Descent

In [21]:
w_gradient = np.zeros(X.shape[1])
b = 0.0
alpha = 0.01
N = len(T)
max_iter = 10000000
i = 0

while True:
    y = np.dot(X_scaled, w_gradient) + b
    dw = (1/N) * np.dot(X_scaled.T, (y - T))
    db = (1/N) * np.sum(y - T)

    temp_w = w_gradient - alpha * dw
    temp_b = b - alpha * db

    if (np.sum(np.abs(temp_w - w_gradient)) < 1e-3 and abs(temp_b - b) < 1e-3):
        break

    
    w_gradient = temp_w
    b = temp_b
    i+=1
    if i >= max_iter:
        print("Stopped: reached maximum iterations without full convergence")
        break

print("Learned Weights using Gradient Descent:", np.hstack((b, w_gradient)))

Learned Weights using Gradient Descent: [207183.64947739  73339.26283801  11390.84621904  -9813.26425025
  39004.27702662 -50396.39066188  25029.65362835 -91055.64221427
 -58467.87257439 -12788.68906468 -34718.63853906  68415.11508797
   7980.48882297]


## Validating Linear Regression Models 

### Applying model on validation dataset

In [16]:
x_val = validation_data.iloc[:, 1:13]
t_val = validation_data.iloc[:, 0]
# For gradient descent solution
x_val_scaled = (x_val - X_mean) / X_std
# For closed-form solution
x_val_biased = np.hstack((np.ones((x_val_scaled.shape[0], 1)), x_val_scaled))

y_val_closed_form = x_val_biased @ W_star
y_val_gradient = np.dot(x_val_scaled, w_gradient) + b

### Error metrics

In [17]:
mse_closed_val = np.mean((y_val_closed_form - t_val.values.flatten())**2)
mse_gradient_val = np.mean((y_val_gradient - t_val.values.flatten())**2)
print("Validation MSE (Closed-form):", mse_closed_val)
print("Validation MSE (Gradient Descent):", mse_gradient_val)

mae_closed_val = np.mean(np.abs(y_val_closed_form - t_val.values.flatten()))
mae_gradient_val = np.mean(np.abs(y_val_gradient - t_val.values.flatten()))
print("Validation MAE (Closed-form):", mae_closed_val)
print("Validation MAE (Gradient Descent):", mae_gradient_val)

Validation MSE (Closed-form): 4583009571.727016
Validation MSE (Gradient Descent): 4585490675.529079
Validation MAE (Closed-form): 49752.37648772887
Validation MAE (Gradient Descent): 49785.30702090824


## Testing Linear Regression models

### Applying test data

In [18]:
x_test = test_data.iloc[:, 1:13]
t_test = test_data.iloc[:, 0]

# For gradient descent solution
x_test_scaled = (x_test - X_mean) / X_std

# For closed-form solution
x_test_biased = np.hstack((np.ones((x_test_scaled.shape[0], 1)), x_test_scaled))

y_test_closed_form = x_test_biased @ W_star
y_test_gradient = np.dot(x_test_scaled, w_gradient) + b

### Error Metrics

In [19]:
mse_closed_val = np.mean((y_test_closed_form - t_test.values.flatten())**2)
mse_gradient_val = np.mean((y_test_gradient - t_test.values.flatten())**2)
print("Test MSE (Closed-form):", mse_closed_val)
print("Test MSE (Gradient Descent):", mse_gradient_val)

mae_closed_val = np.mean(np.abs(y_test_closed_form - t_test.values.flatten()))
mae_gradient_val = np.mean(np.abs(y_test_gradient - t_test.values.flatten()))
print("Test MAE (Closed-form):", mae_closed_val)
print("Test MAE (Gradient Descent):", mae_gradient_val)

Test MSE (Closed-form): 5132832734.377795
Test MSE (Gradient Descent): 5130525026.443078
Test MAE (Closed-form): 50984.58060566474
Test MAE (Gradient Descent): 50979.18897040455


## Ridge Regression (L2 Regularization)

### Applying model

In [20]:
X_biased_scaled = np.hstack((np.ones((X_scaled.shape[0], 1)), X_scaled))
I = np.identity(X_biased_scaled.shape[1])
I[0, 0] = 0 
lambda_values = [0.001 , 0.01 , 0.1, 1, 10, 100]
mse_ridge_val = []
mae_ridge_val = []
w_ridge_array = []
for lambda_reg in lambda_values:
    penalty = lambda_reg * I
    w_ridge = np.linalg.inv(X_biased_scaled.T @ X_biased_scaled + penalty) @ X_biased_scaled.T @ T
    w_ridge_array.append(w_ridge)
    y_val_ridge = x_val_biased @ w_ridge
    mse_ridge = np.mean((y_val_ridge - t_val.values.flatten())**2)
    mae_ridge = np.mean(np.abs(y_val_ridge - t_val.values.flatten()))
    mse_ridge_val.append(mse_ridge)
    mae_ridge_val.append(mae_ridge)
    print(f"Ridge Regression (lambda={lambda_reg}): Validation MSE: {mse_ridge}, MAE: {mae_ridge}")
best_lambda_index = np.argmin(mae_ridge_val)
best_lambda = lambda_values[best_lambda_index]
best_w_ridge = w_ridge_array[best_lambda_index]
print("Best lambda value for Ridge Regression:", best_lambda)
print("Weights for best Ridge Regression model:", best_w_ridge)


Ridge Regression (lambda=0.001): Validation MSE: 4583010041.349514, MAE: 49752.38622248577
Ridge Regression (lambda=0.01): Validation MSE: 4583014269.17596, MAE: 49752.47380962154
Ridge Regression (lambda=0.1): Validation MSE: 4583056667.372729, MAE: 49753.34714726564
Ridge Regression (lambda=1): Validation MSE: 4583491470.689091, MAE: 49761.835063068385
Ridge Regression (lambda=10): Validation MSE: 4588198510.336591, MAE: 49839.89222224875
Ridge Regression (lambda=100): Validation MSE: 4614996721.268295, MAE: 50205.42862480137
Best lambda value for Ridge Regression: 0.001
Weights for best Ridge Regression model: [207183.64947739  73339.22739798  11390.79194217  -9813.06358405
  39003.30943989 -50396.55016377  25030.57476307 -91068.64171939
 -58463.49905727 -12788.49642211 -34720.46616342  68434.47958764
   7981.10325011]


# Final Comment : The dataset doesn't need a L2 Regularization as it can be solved using only Linear regression due to high correlation of data. Speaking of direct solution vs. Gradient descent techniques , we can observe that they are quite similar thanks to the small learning rate of the model. We can shrink it more to gain even smaller gap between them.